In [1]:
import sys
import random 
import requests
import numpy as np

# assumes working directory is notebook location
corepath = "../../../helao-core"
sys.path.append(corepath)
sys.path.append("../..")
from helaocore.schema import Sequence
from helaocore.model.experiment import ExperimentTemplate
import helao.experiment.simulate_exp
from simulate_dev import config as global_cfg

cfg = global_cfg['servers']

# request explorable space

In [2]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/list_all_spaces")
resp.status_code == 200

True

In [3]:
# available composition,pH spaces
sorted([(d['elements'], d['solution_ph']) for d in resp.json()])

[(['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 3),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 7),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 9),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 13),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 3),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 7),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 9),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 13),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 3),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 7),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 9),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 13),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 3),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 7),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 9),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 13)]

### terminology
- a 'sequence' is a queue of experiments performed in order
- an 'experiment' is a queue of actions performed in order
- _side note:_ one or more ESAMP processes may be created from an experiment
- an 'action' is the atomic request dispatched by the Orchestrator to individual action servers
- an 'action server' exposes hardware driver and data management functions via FastAPI
- an 'Orchestrator' manages the queing and dispatch sequences, experiments, and action requests

In [4]:
# valid experiment names
helao.experiment.simulate_exp.__all__

['SIM_measure_CP']

In [5]:
from helao.experiment.simulate_exp import SIM_measure_CP
SIM_measure_CP

<function helao.experiment.simulate_exp.SIM_measure_CP(experiment: helaocore.schema.Experiment, experiment_version: int = 1, solution_ph: Optional[int] = 13, elements: Optional[List[str]] = [], element_fracs: Optional[List[float]] = [])>

- `SIM_measure_CP` is the only experiment available in this simulator
- `SIM_measure_CP` has 3 real arguments: `solution_ph`, `elements`, and `element_fracs`
- args `experiment` and `experiment_version` are managed by orchestrator

# example Ni-Fe-La-Ce-Co-Ta @ pH=13

In [6]:
import json
json.dumps(['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'])

'["Ni", "Fe", "La", "Ce", "Co", "Ta"]'

In [7]:
# get addressable composition space from previous request
elements = ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta']
solution_ph = 13

comp_space = [x for x in resp.json() if x['elements']==elements and x['solution_ph']==solution_ph][0]['element_fracs']
len(comp_space)

2051

In [8]:
comp_space[:5]

[[0.1, 0.2, 0.0, 0.0, 0.6, 0.1],
 [0.0, 0.3, 0.0, 0.2, 0.5, 0.0],
 [0.2, 0.0, 0.1, 0.0, 0.4, 0.3],
 [0.1, 0.0, 0.4, 0.1, 0.4, 0.0],
 [0.2, 0.1, 0.2, 0.0, 0.0, 0.5]]

In [9]:
# initial random seed of compositions
random.seed(0)
comp_inds = random.sample(range(len(comp_space)), 5)

In [10]:
# when HELAO server group is first launched, there is no plate library loaded and therefore no available composition space to query
# the `SIM_measure_CP` experiment performs 9 actions, the first of which is to load a plate library

In [11]:
# create sequence object for holding experiments
sequence = Sequence(sequence_name='seed_sequence')

In [12]:
for i in comp_inds:
    sequence.experiment_plan_list.append(
        ExperimentTemplate(
            experiment_name="SIM_measure_CP",
            experiment_params={
                "solution_ph": 13,
                "elements": ["Ni", "Fe", "La", "Ce", "Co", "Ta"],
                "element_fracs": comp_space[i],
            },
        )
    )

In [13]:
sequence.as_dict()

{'sequence_name': 'seed_sequence',
 'sequence_params': {},
 'sequence_label': 'noLabel',
 'experiment_plan_list': [{'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.1, 0.8, 0.0, 0.0, 0.1]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.3, 0.3, 0.0, 0.4, 0.0]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.0, 0.2, 0.0, 0.2, 0.6]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.0, 0.4, 0.3, 0.2, 0.1]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'L

In [14]:
seq_req = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/append_sequence", json={"sequence": sequence.as_dict()})
seq_req.status_code == 200  # successful post request

True

In [15]:
orch_list = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/list_sequences")
orch_list.status_code == 200  # successful post request

True

In [16]:
orch_list.json() # present sequence queue

[{'sequence_name': 'seed_sequence',
  'sequence_params': {},
  'sequence_label': 'noLabel',
  'experiment_plan_list': [{'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.1, 0.8, 0.0, 0.0, 0.1]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.3, 0.3, 0.0, 0.4, 0.0]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.0, 0.2, 0.0, 0.2, 0.6]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.0, 0.4, 0.3, 0.2, 0.1]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elem

In [17]:
# start Orch (begin or resume dispatching sequence/experiment/action queues)
orch_start = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/start")
orch_start.status_code == 200

True

In [36]:
# check Orch statuses
orch_status = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/get_status")
orch_status.status_code == 200

True

In [37]:
orch_status.json().keys()

dict_keys(['orchestrator', 'active_dict', 'finished_dict', 'loop_intent', 'loop_state', 'orch_state', 'counter_dispatched_actions', 'server_dict'])

In [41]:
list(orch_status.json()['finished_dict']['finished'].items())[0][1]['act']['experiment_uuid']

'9d41876c-3746-486b-bc30-561a106cafe0'

In [42]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
acq_resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/get_measured")
acq_resp.status_code == 200

True

In [43]:
acq_resp.json()

[{'Ni': 0.0,
  'Fe': 0.1,
  'La': 0.8,
  'Ce': 0.0,
  'Co': 0.0,
  'Ta': 0.1,
  'solution_ph': 13,
  'eta3': 0.411175,
  'eta10': 0.502737},
 {'Ni': 0.0,
  'Fe': 0.3,
  'La': 0.3,
  'Ce': 0.0,
  'Co': 0.4,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.361331,
  'eta10': 0.397913},
 {'Ni': 0.0,
  'Fe': 0.0,
  'La': 0.2,
  'Ce': 0.0,
  'Co': 0.2,
  'Ta': 0.6,
  'solution_ph': 13,
  'eta3': 0.41637,
  'eta10': 0.462762},
 {'Ni': 0.0,
  'Fe': 0.0,
  'La': 0.4,
  'Ce': 0.3,
  'Co': 0.2,
  'Ta': 0.1,
  'solution_ph': 13,
  'eta3': 0.395245,
  'eta10': 0.456316},
 {'Ni': 0.2,
  'Fe': 0.0,
  'La': 0.3,
  'Ce': 0.5,
  'Co': 0.0,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.417371,
  'eta10': 0.494323}]